In [147]:
import pandas as pd 
from datetime import datetime

In [148]:
import os
import glob
from dateutil.parser import parse

In [149]:
def convert_mixed_dates(col):
    """Handle Excel ordinal dates and string dates in same column"""
    # First try converting Excel ordinal numbers
    ordinal_dates = pd.to_numeric(col, errors='coerce')
    ordinal_converted = pd.to_datetime(
        ordinal_dates.dropna(),
        unit='D',
        origin='1899-12-30',  # Excel's date system origin
        errors='coerce'
    )
    
    # Then convert string dates with format specification
    string_dates = pd.to_datetime(
        col,
        format='%m/%d/%Y',   # Modify this to match your date format
        errors='coerce'
    )
    
    # Combine results, prioritizing valid ordinal conversions
    return string_dates.fillna(ordinal_converted)

def load_data_with_mixed_dates(root_folder):
    combined_data = pd.DataFrame()
    
    for root, _, files in os.walk(root_folder):
        for filename in files:
            if filename.endswith('.xlsx'):
                file_path = os.path.join(root, filename)
                try:
                    df = pd.read_excel(file_path, sheet_name='Worksheet', header=None)
                    
                    # Extract maturity date from row 1 column 4
                    maturity_str = df.iloc[0, 3]
                    maturity_date = pd.to_datetime(maturity_str, errors='coerce')
                    if pd.isna(maturity_date):
                        print(f"Skipping {filename}: Invalid maturity date format")
                        continue
                    month_year = maturity_date.strftime("%b%Y")

                    # Process data starting from row 6
                    data = df.iloc[5:].copy()
                    data.columns = df.iloc[5].tolist()
                    
                    if 'Date' not in data.columns:
                        print(f"Skipping {filename}: Missing Date column")
                        continue

                    # Convert dates with mixed format handling
                    data['Date'] = convert_mixed_dates(data['Date'])
                    
                    # Clean and validate
                    invalid_dates = data['Date'].isna()
                    if invalid_dates.any():
                        print(f"Found {invalid_dates.sum()} invalid dates in {filename}")
                        data = data[~invalid_dates]
                    
                    data = data[['Date', 'PX_LAST', 'YLD_YTM_MID']]
                    data.columns = ['Date', f'Price_{month_year}', f'Yield_{month_year}']
                    
                    # Merge data
                    combined_data = pd.merge(
                        combined_data, 
                        data, 
                        on='Date', 
                        how='outer'
                    ) if not combined_data.empty else data

                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
    
    return combined_data.sort_values('Date').dropna(how='all', axis=1).reset_index(drop=True)


In [150]:
combined_data = load_data_with_mixed_dates('Principal Strips')

Found 1 invalid dates in grid1_dgxs1n52.xlsx
Found 1 invalid dates in grid1_2vzqrtu3.xlsx
Found 1 invalid dates in grid1_px2bq3zf.xlsx
Found 1 invalid dates in grid1_ukwjkwb1.xlsx
Found 1 invalid dates in grid1.xlsx
Found 1 invalid dates in grid1_orztzzic.xlsx
Found 1 invalid dates in grid1_bvwjfxia.xlsx
Found 1 invalid dates in grid1_ed5upmya.xlsx
Found 1 invalid dates in grid1_30rtdskq.xlsx
Found 1 invalid dates in grid1_4q5ugdeu.xlsx
Found 1 invalid dates in grid1_b2mccazu.xlsx
Found 1 invalid dates in grid1_dddkxdn2.xlsx
Found 1 invalid dates in grid1_ndy0ppu5.xlsx
Found 1 invalid dates in grid1_sm0iv21a.xlsx
Found 1 invalid dates in grid1_uvoqinsq.xlsx
Found 1 invalid dates in grid1_5bbkirvh.xlsx
Found 1 invalid dates in grid1_exmcpyfx.xlsx
Found 1 invalid dates in grid1_xo3vagj3.xlsx
Found 1 invalid dates in grid1_gukg4afy.xlsx
Found 1 invalid dates in grid1_fskw1muf.xlsx
Found 1 invalid dates in grid1_1o4ttufo.xlsx
Found 1 invalid dates in grid1_ihntrvb3.xlsx
Found 1 invalid dat

In [151]:
def sort_combined_data(combined_df):
    """Sort columns by maturity date while keeping Date first"""
    # Extract maturity info from column names
    maturity_info = []
    for col in combined_df.columns:
        if col == 'Date':
            continue
        if '_' in col:
            _, monthyear = col.split('_')
            maturity_date = pd.to_datetime(f"15-{monthyear}", format='%d-%b%Y')
            maturity_info.append((col, maturity_date))
    
    # Sort columns by maturity date
    sorted_cols = sorted(maturity_info, key=lambda x: x[1])
    
    # Create new column order (Date first, then sorted columns)
    new_order = ['Date'] + [col[0] for col in sorted_cols]
    
    return combined_df[new_order]

In [152]:
sorted_combined_data = sort_combined_data(combined_data)

In [153]:
sorted_combined_data.head()

,Date,Price_Feb2025,Yield_Feb2025,Price_May2025,Yield_May2025,Price_Aug2025,Yield_Aug2025,Price_Nov2025,Yield_Nov2025,Price_Feb2026,...,Price_Feb2038,Yield_Feb2038,Price_May2038,Yield_May2038,Price_Feb2039,Yield_Feb2039,Price_May2039,Yield_May2039,Price_Aug2039,Yield_Aug2039
0,2024-01-02,95.289,4.368,93.992,4.59,93.284,4.347,92.273,4.358,91.41,...,58.211,3.87,57.444,3.897,55.291,3.959,54.632,3.974,54.028,3.982
1,2024-01-03,95.288,4.379,94.002,4.591,93.348,4.311,92.266,4.369,91.409,...,58.192,3.873,57.43,3.899,55.281,3.961,54.609,3.977,54,3.986
2,2024-01-04,95.302,4.377,93.955,4.638,93.251,4.384,92.199,4.415,91.308,...,57.458,3.966,56.712,3.989,54.534,4.053,53.855,4.07,53.239,4.079
3,2024-01-05,NaN,NaN,93.988,4.64,93.261,4.4,92.223,4.421,91.351,...,57.021,4.023,56.266,4.048,54.09,4.111,53.393,4.13,52.786,4.137
4,2024-01-08,95.316,4.407,94.018,4.626,93.321,4.366,92.262,4.404,91.377,...,57.158,4.007,56.395,4.032,54.216,4.096,53.536,4.113,52.917,4.122


In [154]:
sorted_combined_data = sorted_combined_data.sort_values('Date')

In [155]:
sorted_combined_data = sorted_combined_data.infer_objects(copy=False)

In [156]:
for col in sorted_combined_data.columns:
    if col.startswith('Price_') or col.startswith('Yield_'):
        sorted_combined_data[col] = sorted_combined_data[col].astype('float64').ffill()

In [157]:
sorted_combined_data.tail()

,Date,Price_Feb2025,Yield_Feb2025,Price_May2025,Yield_May2025,Price_Aug2025,Yield_Aug2025,Price_Nov2025,Yield_Nov2025,Price_Feb2026,...,Price_Feb2038,Yield_Feb2038,Price_May2038,Yield_May2038,Price_Feb2039,Yield_Feb2039,Price_May2039,Yield_May2039,Price_Aug2039,Yield_Aug2039
280,2025-01-29,99.505,4.159,98.795,4.205,97.544,3.694,96.756,4.218,95.880,...,55.442,4.574,54.828,4.574,52.632,4.623,51.695,4.671,50.995,4.685
281,2025-01-30,99.505,4.159,98.805,4.211,97.544,3.694,96.770,4.215,95.898,...,55.587,4.554,55.034,4.546,52.787,4.603,51.850,4.651,51.158,4.663
282,2025-01-31,99.505,4.159,98.836,4.221,97.544,3.694,96.799,4.220,95.896,...,55.364,4.589,54.795,4.582,52.547,4.639,51.599,4.688,50.902,4.701
283,2025-02-03,99.505,4.159,98.844,4.235,97.544,3.694,96.801,4.233,95.886,...,55.321,4.595,54.720,4.594,52.488,4.648,51.562,4.694,50.858,4.708
284,2025-02-04,99.932,2.520,98.863,4.204,97.544,3.694,96.805,4.243,95.935,...,55.667,4.548,55.083,4.544,52.863,4.597,51.926,4.645,51.226,4.658


In [158]:
#Convert yields to decimal
for col in sorted_combined_data.columns:
    if col.startswith('Yield'):
        sorted_combined_data[col] = sorted_combined_data[col]/100

In [159]:
sorted_combined_data.head()

,Date,Price_Feb2025,Yield_Feb2025,Price_May2025,Yield_May2025,Price_Aug2025,Yield_Aug2025,Price_Nov2025,Yield_Nov2025,Price_Feb2026,...,Price_Feb2038,Yield_Feb2038,Price_May2038,Yield_May2038,Price_Feb2039,Yield_Feb2039,Price_May2039,Yield_May2039,Price_Aug2039,Yield_Aug2039
0,2024-01-02,95.289,0.04368,93.992,0.04590,93.284,0.04347,92.273,0.04358,91.410,...,58.211,0.03870,57.444,0.03897,55.291,0.03959,54.632,0.03974,54.028,0.03982
1,2024-01-03,95.288,0.04379,94.002,0.04591,93.348,0.04311,92.266,0.04369,91.409,...,58.192,0.03873,57.430,0.03899,55.281,0.03961,54.609,0.03977,54.000,0.03986
2,2024-01-04,95.302,0.04377,93.955,0.04638,93.251,0.04384,92.199,0.04415,91.308,...,57.458,0.03966,56.712,0.03989,54.534,0.04053,53.855,0.04070,53.239,0.04079
3,2024-01-05,95.302,0.04377,93.988,0.04640,93.261,0.04400,92.223,0.04421,91.351,...,57.021,0.04023,56.266,0.04048,54.090,0.04111,53.393,0.04130,52.786,0.04137
4,2024-01-08,95.316,0.04407,94.018,0.04626,93.321,0.04366,92.262,0.04404,91.377,...,57.158,0.04007,56.395,0.04032,54.216,0.04096,53.536,0.04113,52.917,0.04122


In [160]:
# Extract unique maturity dates from column names
maturity_dates = {}
for col in sorted_combined_data.columns:
    if col.startswith('Price_') or col.startswith('Yield_'):
        _, month_year = col.split('_')
        if len(month_year) == 6:  # Format: 'MmmYY'
            year = int(month_year[-2:])
            year = 2000 + year if year < 50 else 1900 + year  # Adjust for century
            maturity_dates[month_year] = pd.Timestamp(f'{year}-{month_year[:3]}-15')
        elif len(month_year) == 7:  # Format: 'MmmYYYY'
            maturity_dates[month_year] = pd.Timestamp(f'{month_year[-4:]}-{month_year[:3]}-15')

# Calculate Time-to-Maturity for all maturities
time_to_maturity = {}
for month_year, maturity_date in maturity_dates.items():
    time_to_maturity[f'Time_{month_year}'] = (maturity_date - sorted_combined_data['Date']).dt.days / 365.25
    # Add a check to ensure no negative values
    time_to_maturity[f'Time_{month_year}'] = time_to_maturity[f'Time_{month_year}'].clip(lower=0)

# Create a new DataFrame with Time-to-Maturity columns
time_to_maturity_df = pd.DataFrame(time_to_maturity)

# Concatenate the original DataFrame with the new Time-to-Maturity columns
sorted_combined_data = pd.concat([sorted_combined_data, time_to_maturity_df], axis=1)

# Fill NaN values for Price and Yield columns with forward fill
for col in sorted_combined_data.columns:
    if col.startswith('Price_') or col.startswith('Yield_'):
        sorted_combined_data[col] = sorted_combined_data[col].interpolate().ffill().bfill()

# Reorder columns to group Price, Time, and Yield for each maturity
columns = ['Date']
for month_year in sorted(maturity_dates.keys(), key=lambda x: maturity_dates[x]):
    for prefix in ['Price_', 'Time_', 'Yield_']:
        col_name = f'{prefix}{month_year}'
        if col_name in sorted_combined_data.columns:
            columns.append(col_name)

# Ensure all original columns are included
missing_columns = set(sorted_combined_data.columns) - set(columns)
columns.extend(missing_columns)

sorted_combined_data = sorted_combined_data[columns]

In [161]:
sorted_combined_data.tail()

,Date,Price_Feb2025,Time_Feb2025,Yield_Feb2025,Price_May2025,Time_May2025,Yield_May2025,Price_Aug2025,Time_Aug2025,Yield_Aug2025,...,Yield_May2038,Price_Feb2039,Time_Feb2039,Yield_Feb2039,Price_May2039,Time_May2039,Yield_May2039,Price_Aug2039,Time_Aug2039,Yield_Aug2039
280,2025-01-29,99.505,0.046543,0.04159,98.795,0.290212,0.04205,97.544,0.542094,0.03694,...,0.04574,52.632,14.045175,0.04623,51.695,14.288843,0.04671,50.995,14.540726,0.04685
281,2025-01-30,99.505,0.043806,0.04159,98.805,0.287474,0.04211,97.544,0.539357,0.03694,...,0.04546,52.787,14.042437,0.04603,51.850,14.286105,0.04651,51.158,14.537988,0.04663
282,2025-01-31,99.505,0.041068,0.04159,98.836,0.284736,0.04221,97.544,0.536619,0.03694,...,0.04582,52.547,14.039699,0.04639,51.599,14.283368,0.04688,50.902,14.535250,0.04701
283,2025-02-03,99.505,0.032854,0.04159,98.844,0.276523,0.04235,97.544,0.528405,0.03694,...,0.04594,52.488,14.031485,0.04648,51.562,14.275154,0.04694,50.858,14.527036,0.04708
284,2025-02-04,99.932,0.030116,0.02520,98.863,0.273785,0.04204,97.544,0.525667,0.03694,...,0.04544,52.863,14.028747,0.04597,51.926,14.272416,0.04645,51.226,14.524298,0.04658


In [162]:
#Save to CSV
STRIPS_data_file = 'STRIPS_data.csv'
sorted_combined_data.to_csv(STRIPS_data_file, index=False)

sorted_combined_data.head()

,Date,Price_Feb2025,Time_Feb2025,Yield_Feb2025,Price_May2025,Time_May2025,Yield_May2025,Price_Aug2025,Time_Aug2025,Yield_Aug2025,...,Yield_May2038,Price_Feb2039,Time_Feb2039,Yield_Feb2039,Price_May2039,Time_May2039,Yield_May2039,Price_Aug2039,Time_Aug2039,Yield_Aug2039
0,2024-01-02,95.289,1.122519,0.04368,93.992,1.366188,0.04590,93.284,1.618070,0.04347,...,0.03897,55.291,15.121150,0.03959,54.632,15.364819,0.03974,54.028,15.616701,0.03982
1,2024-01-03,95.288,1.119781,0.04379,94.002,1.363450,0.04591,93.348,1.615332,0.04311,...,0.03899,55.281,15.118412,0.03961,54.609,15.362081,0.03977,54.000,15.613963,0.03986
2,2024-01-04,95.302,1.117043,0.04377,93.955,1.360712,0.04638,93.251,1.612594,0.04384,...,0.03989,54.534,15.115674,0.04053,53.855,15.359343,0.04070,53.239,15.611225,0.04079
3,2024-01-05,95.302,1.114305,0.04377,93.988,1.357974,0.04640,93.261,1.609856,0.04400,...,0.04048,54.090,15.112936,0.04111,53.393,15.356605,0.04130,52.786,15.608487,0.04137
4,2024-01-08,95.316,1.106092,0.04407,94.018,1.349760,0.04626,93.321,1.601643,0.04366,...,0.04032,54.216,15.104723,0.04096,53.536,15.348392,0.04113,52.917,15.600274,0.04122
